![](./images/08_00.jpg)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.my_pyspark import *

In [3]:
spark = MyPySpark(session=True)

# 1. Đọc dữ liệu

In [4]:
college_path = r"./data/College.csv"

In [5]:
college_data = spark.readFile(college_path)

In [6]:
college_data.show(3, False, True)

-RECORD 0-----------------------------------
 School      | Abilene Christian University 
 Private     | Yes                          
 Apps        | 1660                         
 Accept      | 1232                         
 Enroll      | 721                          
 Top10perc   | 23                           
 Top25perc   | 52                           
 F_Undergrad | 2885                         
 P_Undergrad | 537                          
 Outstate    | 7440                         
 Room_Board  | 3300                         
 Books       | 450                          
 Personal    | 2200                         
 PhD         | 70                           
 Terminal    | 78                           
 S_F_Ratio   | 18.1                         
 perc_alumni | 12                           
 Expend      | 7041                         
 Grad_Rate   | 60                           
-RECORD 1-----------------------------------
 School      | Adelphi University           
 Private  

In [7]:
college_data.describe().toPandas()

,summary,School,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F_Undergrad,P_Undergrad,Outstate,Room_Board,Books,Personal,PhD,Terminal,S_F_Ratio,perc_alumni,Expend,Grad_Rate
0,count,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777
1,mean,None,None,3001.6383526383524,2018.8043758043757,779.972972972973,27.55855855855856,55.7966537966538,3699.907335907336,855.2985842985843,10440.66924066924,4357.526383526383,549.3809523809524,1340.6422136422136,72.66023166023166,79.70270270270271,14.089703989703986,22.743886743886744,9660.17117117117,65.46332046332046
2,stddev,None,None,3870.2014844352884,2451.11397099263,929.17619013287,17.640364385452134,19.804777595131373,4850.420530887386,1522.431887295513,4023.0164841119727,1096.6964155935289,165.10536013709253,677.071453590578,16.328154687939314,14.722358527903374,3.9583491352055478,12.39180148937615,5221.76843985609,17.177709897155403
3,min,Abilene Christian University,No,81,72,35,1,9,139,1,2340,1780,96,250,8,24,2.5,0,3186,10
4,max,York College of Pennsylvania,Yes,48094,26330,6392,96,100,31643,21836,21700,8124,2340,6800,103,100,39.8,64,56233,118


In [8]:
college_data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



# 2. Chuẩn dữ liệu
## 2.1. Indexer feature `Private`

In [9]:
from modules.my_spark_preprocessing import *

In [10]:
college_data = indexerData(college_data, ['Private'])

In [11]:
college_data.show(1, False, True)

-RECORD 0-----------------------------------
 School      | Abilene Christian University 
 Private     | Yes                          
 Apps        | 1660                         
 Accept      | 1232                         
 Enroll      | 721                          
 Top10perc   | 23                           
 Top25perc   | 52                           
 F_Undergrad | 2885                         
 P_Undergrad | 537                          
 Outstate    | 7440                         
 Room_Board  | 3300                         
 Books       | 450                          
 Personal    | 2200                         
 PhD         | 70                           
 Terminal    | 78                           
 S_F_Ratio   | 18.1                         
 perc_alumni | 12                           
 Expend      | 7041                         
 Grad_Rate   | 60                           
 idx_Private | 0.0                          
only showing top 1 row



# 3. Chuyển đổi dữ liệu

In [12]:
predictor_vars = ['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad', 'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate']
target_var = 'idx_Private'

In [13]:
data_handler = MyDataClassifier()
data_handler.initModel(college_data, predictor_vars, target_var)

In [14]:
data_handler.prepareData()

# 4. Tách dữ liệu thành train và test

In [15]:
data_handler.trainTestSplit(.7)

# 5. Build Model

In [16]:
from modules.my_decision_tree import *
from modules.my_random_forest import *
from modules.my_gbt import *

In [17]:
dtree_model = MyDecisionTree(data_handler)
rdfor_model = MyRandomForest(data_handler)
gbt_model = MyGBT(data_handler)

In [18]:
dtree_model.build()
rdfor_model.build()
gbt_model.build()

In [19]:
dtree_test_predict = dtree_model.predictTestData()
rdfor_test_predict = rdfor_model.predictTestData()
gbt_test_predict = gbt_model.predictTestData()

In [20]:
dtree_test_predict.show(3, False, True)

-RECORD 0------------------------------------------------------------------------------------------------------------
 features      | [212.0,197.0,91.0,28.0,56.0,471.0,148.0,13500.0,5230.0,400.0,850.0,95.0,98.0,9.3,37.0,16095.0,52.0] 
 idx_Private   | 0.0                                                                                                 
 rawPrediction | [234.0,0.0]                                                                                         
 probability   | [1.0,0.0]                                                                                           
 prediction    | 0.0                                                                                                 
-RECORD 1------------------------------------------------------------------------------------------------------------
 features      | [213.0,155.0,75.0,28.0,66.0,504.0,147.0,7200.0,3532.0,550.0,1500.0,56.0,56.0,11.8,12.0,7818.0,52.0] 
 idx_Private   | 0.0                                    

In [21]:
rdfor_test_predict.show(3, False, True)

-RECORD 0------------------------------------------------------------------------------------------------------------
 features      | [212.0,197.0,91.0,28.0,56.0,471.0,148.0,13500.0,5230.0,400.0,850.0,95.0,98.0,9.3,37.0,16095.0,52.0] 
 idx_Private   | 0.0                                                                                                 
 rawPrediction | [19.850498921150066,0.1495010788499337]                                                             
 probability   | [0.9925249460575033,0.007475053942496685]                                                           
 prediction    | 0.0                                                                                                 
-RECORD 1------------------------------------------------------------------------------------------------------------
 features      | [213.0,155.0,75.0,28.0,66.0,504.0,147.0,7200.0,3532.0,550.0,1500.0,56.0,56.0,11.8,12.0,7818.0,52.0] 
 idx_Private   | 0.0                                    

In [22]:
gbt_test_predict.show(3, False, True)

-RECORD 0------------------------------------------------------------------------------------------------------------
 features      | [212.0,197.0,91.0,28.0,56.0,471.0,148.0,13500.0,5230.0,400.0,850.0,95.0,98.0,9.3,37.0,16095.0,52.0] 
 idx_Private   | 0.0                                                                                                 
 rawPrediction | [1.5495285788155515,-1.5495285788155515]                                                            
 probability   | [0.9568538369777853,0.043146163022214745]                                                           
 prediction    | 0.0                                                                                                 
-RECORD 1------------------------------------------------------------------------------------------------------------
 features      | [213.0,155.0,75.0,28.0,66.0,504.0,147.0,7200.0,3532.0,550.0,1500.0,56.0,56.0,11.8,12.0,7818.0,52.0] 
 idx_Private   | 0.0                                    

In [24]:
dtree_model.evaluate()

,Metric,Values
0,Accuracy,0.897778


In [25]:
rdfor_model.evaluate()

,Metric,Values
0,Accuracy,0.96


In [26]:
gbt_model.evaluate()

,Metric,Values
0,Accuracy,0.902222


> **Nhận xét**
> * Random forest cho ra kết quả tốt nhất